In [38]:
import spark.sql

import spark.sql


In [2]:
val ds1 = spark.range(1, 10000000) 
val ds2 = spark.range(1, 10000000, 2) 
val ds3 = ds1.repartition(7) 
val ds4 = ds2.repartition(9) 

ds1: org.apache.spark.sql.Dataset[Long] = [id: bigint]
ds2: org.apache.spark.sql.Dataset[Long] = [id: bigint]
ds3: org.apache.spark.sql.Dataset[Long] = [id: bigint]
ds4: org.apache.spark.sql.Dataset[Long] = [id: bigint]


In [13]:
val ds5 = ds3.selectExpr("id * 5 as id")
val joined = ds5.join(ds4, "id") 
val sum = joined.selectExpr("sum(id)")

ds5: org.apache.spark.sql.DataFrame = [id: bigint]
joined: org.apache.spark.sql.DataFrame = [id: bigint]
sum: org.apache.spark.sql.DataFrame = [sum(id): bigint]


In [17]:
 sum.explain(extended = true)

== Parsed Logical Plan ==
'Project [unresolvedalias('sum('id), Some(org.apache.spark.sql.Column$$Lambda$2846/1276057405@5298607))]
+- Project [id#20L]
   +- Join Inner, (id#20L = id#2L)
      :- Project [(id#0L * cast(5 as bigint)) AS id#20L]
      :  +- Repartition 7, true
      :     +- Range (1, 10000000, step=1, splits=Some(4))
      +- Repartition 9, true
         +- Range (1, 10000000, step=2, splits=Some(4))

== Analyzed Logical Plan ==
sum(id): bigint
Aggregate [sum(id#20L) AS sum(id)#24L]
+- Project [id#20L]
   +- Join Inner, (id#20L = id#2L)
      :- Project [(id#0L * cast(5 as bigint)) AS id#20L]
      :  +- Repartition 7, true
      :     +- Range (1, 10000000, step=1, splits=Some(4))
      +- Repartition 9, true
         +- Range (1, 10000000, step=2, splits=Some(4))

== Optimized Logical Plan ==
Aggregate [sum(id#20L) AS sum(id)#24L]
+- Project [id#20L]
   +- Join Inner, (id#20L = id#2L)
      :- Project [(id#0L * 5) AS id#20L]
      :  +- Repartition 7, true
      :     

In [21]:
sum.show(5)

+-------------+
|      sum(id)|
+-------------+
|5000000000000|
+-------------+



In [23]:
sum.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (15)
+- HashAggregate (14)
   +- Exchange (13)
      +- HashAggregate (12)
         +- Project (11)
            +- SortMergeJoin Inner (10)
               :- Sort (5)
               :  +- Exchange (4)
               :     +- Project (3)
               :        +- Exchange (2)
               :           +- Range (1)
               +- Sort (9)
                  +- Exchange (8)
                     +- Exchange (7)
                        +- Range (6)


(1) Range
Output [1]: [id#0L]
Arguments: Range (1, 10000000, step=1, splits=Some(4))

(2) Exchange
Input [1]: [id#0L]
Arguments: RoundRobinPartitioning(7), REPARTITION_BY_NUM, [plan_id=84]

(3) Project
Output [1]: [(id#0L * 5) AS id#20L]
Input [1]: [id#0L]

(4) Exchange
Input [1]: [id#20L]
Arguments: hashpartitioning(id#20L, 200), ENSURE_REQUIREMENTS, [plan_id=94]

(5) Sort
Input [1]: [id#20L]
Arguments: [id#20L ASC NULLS FIRST], false, 0

(6) Range
Output [1]: [id#2L]
Arguments: Range (1, 10000000, ste

In [66]:
spark.conf.set("spark.sql.inMemoryColumnarStorage.compressed",false);

In [48]:
spark.conf.get("spark.sql.inMemoryColumnarStorage.compressed")

res17: String = SET spark.sql.inMemoryColumnarStorage.compressed=false


In [68]:
spark.conf.set("spark.sql.inMemoryColumnarStorage.compressed",true)

In [90]:
from spark import SparkConf
conf = SparkConf()
conf.get("spark.sql.serializer")

<console>: 2: error: ';' expected but 'import' found.